In [2]:
# Change this line if you're using Colab to something like '/content/drive/MyDrive/TeamX/'
# where TeamX is just the clone of repository on your Google Drive
# and you have mounted the drive at /content/drive  
# See the Tutorial Slides for more detail.

# Works on your local machine but not on Colab!
PROJECT_ROOT = '../..' 

# Fix this path and use this one on Colab
# PROJECT_ROOT = '/content/drive/MyDrive/TeamX'

import sys
from os.path import join as ospj
sys.path.append(ospj(PROJECT_ROOT, 'src'))

%load_ext autoreload
%autoreload 2

In [3]:
""" Testing the dataset by displaying the scene and the corresponding poses directly from image and annotation files.
"""

import os
import cv2
import numpy as np

from utils.transforms import sample
from utils.viz import annotate

seq_name = "courtyard_arguing_00"
seq_dir = ospj(PROJECT_ROOT,"data", "3DPWPreprocessed", "sequenceFiles")
img_dir = ospj(PROJECT_ROOT,"data", "3DPWPreprocessed", "imageFiles")
folder = "train"
seq_path = ospj(seq_dir, folder, seq_name + ".npz")

seq = np.load(seq_path, allow_pickle=True)

poses = seq["abs_poses"]

trans = seq["trans"]
jointPositions = seq["jointPositions"]
cam_ext = seq["extrinsics"]
cam_int = seq["intrinsics"]
pmask = seq["pmask"]
img_dir = os.path.join(img_dir, seq_name)    
img_names = os.listdir(img_dir)

cv2.namedWindow(f"Padded Resized Image", cv2.WINDOW_KEEPRATIO)  
cv2.namedWindow(f"Padding Mask", cv2.WINDOW_KEEPRATIO)  
print(cam_ext.shape)
print(poses.shape)
for id in range(poses.shape[1]):
    img, _, root_joint, norm_pose, abs_pose = sample(id, poses, img_names, img_dir,  imsize=(384, 384, 3))    
    img = annotate(img, poses[:, id, :, :], root_joint)
    cv2.imshow("Padded Resized Image", img)
    cv2.imshow("Padding Mask", pmask)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break    



cv2.destroyAllWindows()

(765, 4, 4)
(2, 765, 18, 3)


In [9]:
""" Testing the dataset by displaying the scene and the corresponding poses from TFRecords.
"""

import cv2
import numpy as np

from data.threeDPW import ThreeDPWTFRecordDataset
from utils.transforms import cvt_absolute_pose
from utils.viz import annotate_pose, annotate_root

tfrecord_path = ospj(PROJECT_ROOT,"data", "3DPWPreprocessedTFRecord", "train.tfrecord")

dataset = ThreeDPWTFRecordDataset(tfrecord_path, history_window=15, future_window=15, batch_size=1)

print(f"Length of dataset: {len(dataset)}")

loader = dataset.get_loader()

for i, (history, future) in enumerate(loader):
    img = history[0][0][0].numpy()
    norm_pose = history[1][0][0].numpy()
    root_joint = history[2][0][0].numpy()
    mask = history[3][0].numpy()
    annoted_img = annotate_pose(img=img, pose=abs_pose, color=(255, 0, 0), radius=2, thickness=3, text=False)
    annoted_img = annotate_root(img=annoted_img,root=np.expand_dims(root_joint, 0), color=(0, 255, 0))
    cv2.imshow("History Image", img)
    cv2.imshow("History Mask", mask*255)
    
    norm_pose = future[0][0][0].numpy()
    root_joint = future[1][0][0].numpy()
    abs_pose = cvt_absolute_pose(root_joint=np.expand_dims(root_joint, 0), norm_pose=np.expand_dims(norm_pose, 0))
    annoted_img = annotate_pose(img=img, pose=abs_pose, color=(0, 255, 0), radius=2, thickness=3, text=False)
    annoted_img = annotate_root(img=annoted_img,root=np.expand_dims(root_joint, 0), color=(0, 255, 0))
    cv2.imshow("Future Image", img)
    cv2.imshow("Future Mask", mask*255)
    
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break     

cv2.destroyAllWindows()

Length of dataset: 4648


TypeError: annotate_root() got an unexpected keyword argument 'text'